##### Copyright 2022 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/tutorials/video/transfer_learning_with_movinet"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/tutorials/video/transfer_learning_with_movinet.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/tutorials/video/transfer_learning_with_movinet.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/tutorials/video/transfer_learning_with_movinet.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

# Aprendizado por transferência para a classificação de vídeos com MoViNet

O MoViNets (Mobile Video Networks – redes de vídeo para dispositivos móveis) oferece uma família de modelos eficientes para a classificação de vídeos, com suporte à inferência de vídeos de streaming. Neste tutorial, usaremos um modelo MoViNet treinado para classificar vídeos, especificamente para uma tarefa de reconhecimento de ações, do [dataset UCF101](https://www.crcv.ucf.edu/data/UCF101.php). Um modelo pré-treinado é uma rede salva treinada anteriormente com um dataset grande. Confira mais detalhes sobre o MoViNets no artigo [MoViNets: Redes de vídeo para dispositivos móveis para reconhecimento eficiente de vídeos](https://arxiv.org/abs/2103.11511) escrito por Kondratyuk, D. et al. (2021). Neste tutorial, você vai:

- Aprender a baixar um modelo MoViNet pré-treinado.
- Criar um novo modelo usando um modelo pré-treinado com um novo classificador por meio do congelamento da base convolucional do modelo MoViNet.
- Substituir o cabeçalho do classificador pelo número de rótulos de um novo dataset.
- Fazer o aprendizado por transferência para o [dataset UCF101](https://www.crcv.ucf.edu/data/UCF101.php).

O modelo baixado neste tutorial veio de [official/projects/movinet](https://github.com/tensorflow/models/tree/master/official/projects/movinet). Esse repositório contém uma coleção de modelos MoViNet que o TF Hub usa no formato SavedModel do TensorFlow 2.

Este tutorial de aprendizado por transferência é a terceira parte de uma série de tutoriais do TensorFlow sobre vídeos. Aqui estão os outros três tutoriais:

- [Carregue dados de vídeo](https://www.tensorflow.org/tutorials/load_data/video): este tutorial explica boa parte do código usado neste documento, em especial, é explicado mais detalhadamente como pré-processar e carregar os dados usando a classe `FrameGenerator`.
- [Crie um modelo CNN 3D para a classificação de vídeos](https://www.tensorflow.org/tutorials/video/video_classification): observe que este tutorial usa uma CNN (2+1)D que decompõe os aspectos espaciais e temporais dos dados 3D. Se você estiver usando dados volumétricos, como uma ressonância magnética, considere usar uma CNN 3D em vez de uma CNN (2+1)D.
- [MoViNet para o reconhecimento de ações de streaming](https://www.tensorflow.org/hub/tutorials/movinet): conheça os modelos MoViNet disponíveis no TF Hub.

## Configuração

Comece instalando e importando algumas bibliotecas necessárias, incluindo: [remotezip](https://github.com/gtsystem/python-remotezip) para analisar o conteúdo de um arquivo ZIP, [tqdm](https://github.com/tqdm/tqdm) para usar uma barra de progresso, [OpenCV](https://opencv.org/) para processar arquivos de vídeo (`opencv-python` e `opencv-python-headless` precisam estar na mesma versão) e modelos do TensorFlow ([`tf-models-official`](https://github.com/tensorflow/models/tree/master/official)) para baixar o modelo MoViNet pré-treinado. O pacote de modelos do TensorFlow é uma coleção de modelos que usam as APIs de alto nível do TensorFlow.

In [ ]:
!pip install remotezip tqdm opencv-python==4.5.2.52 opencv-python-headless==4.5.2.52 tf-models-official

In [ ]:
import tqdm
import random
import pathlib
import itertools
import collections

import cv2
import numpy as np
import remotezip as rz
import seaborn as sns
import matplotlib.pyplot as plt

import keras
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

# Import the MoViNet model from TensorFlow Models (tf-models-official) for the MoViNet model
from official.projects.movinet.modeling import movinet
from official.projects.movinet.modeling import movinet_model

## Carregue os dados

A célula oculta abaixo define funções helper para baixar um segmento dos dados do dataset UCF-101 e carregá-los em um `tf.data.Dataset`. O [tutorial Carregue dados de vídeos](https://www.tensorflow.org/tutorials/load_data/video) apresenta instruções detalhadas sobre esse código.

A classe `FrameGenerator` no final do bloco oculto é o utilitário mais importante que temos aqui, pois cria um objeto iterável que pode alimentar dados no pipeline de dados do TensorFlow. Especificamente, essa classe contém um gerador Python que carrega os quadros do vídeo juntamente com seu rótulo codificado. A função geradora (`__call__`) gera a array de quadros produzida por `frames_from_video_file` e um vetor com codificação one-hot do rótulo associado ao conjunto de quadros.


In [ ]:
#@title 

def list_files_per_class(zip_url):
  """
    List the files in each class of the dataset given the zip URL.

    Args:
      zip_url: URL from which the files can be unzipped. 

    Return:
      files: List of files in each of the classes.
  """
  files = []
  with rz.RemoteZip(URL) as zip:
    for zip_info in zip.infolist():
      files.append(zip_info.filename)
  return files

def get_class(fname):
  """
    Retrieve the name of the class given a filename.

    Args:
      fname: Name of the file in the UCF101 dataset.

    Return:
      Class that the file belongs to.
  """
  return fname.split('_')[-3]

def get_files_per_class(files):
  """
    Retrieve the files that belong to each class. 

    Args:
      files: List of files in the dataset.

    Return:
      Dictionary of class names (key) and files (values).
  """
  files_for_class = collections.defaultdict(list)
  for fname in files:
    class_name = get_class(fname)
    files_for_class[class_name].append(fname)
  return files_for_class

def download_from_zip(zip_url, to_dir, file_names):
  """
    Download the contents of the zip file from the zip URL.

    Args:
      zip_url: Zip URL containing data.
      to_dir: Directory to download data to.
      file_names: Names of files to download.
  """
  with rz.RemoteZip(zip_url) as zip:
    for fn in tqdm.tqdm(file_names):
      class_name = get_class(fn)
      zip.extract(fn, str(to_dir / class_name))
      unzipped_file = to_dir / class_name / fn

      fn = pathlib.Path(fn).parts[-1]
      output_file = to_dir / class_name / fn
      unzipped_file.rename(output_file,)

def split_class_lists(files_for_class, count):
  """
    Returns the list of files belonging to a subset of data as well as the remainder of
    files that need to be downloaded.

    Args:
      files_for_class: Files belonging to a particular class of data.
      count: Number of files to download.

    Return:
      split_files: Files belonging to the subset of data.
      remainder: Dictionary of the remainder of files that need to be downloaded.
  """
  split_files = []
  remainder = {}
  for cls in files_for_class:
    split_files.extend(files_for_class[cls][:count])
    remainder[cls] = files_for_class[cls][count:]
  return split_files, remainder

def download_ufc_101_subset(zip_url, num_classes, splits, download_dir):
  """
    Download a subset of the UFC101 dataset and split them into various parts, such as
    training, validation, and test. 

    Args:
      zip_url: Zip URL containing data.
      num_classes: Number of labels.
      splits: Dictionary specifying the training, validation, test, etc. (key) division of data 
              (value is number of files per split).
      download_dir: Directory to download data to.

    Return:
      dir: Posix path of the resulting directories containing the splits of data.
  """
  files = list_files_per_class(zip_url)
  for f in files:
    tokens = f.split('/')
    if len(tokens) <= 2:
      files.remove(f) # Remove that item from the list if it does not have a filename

  files_for_class = get_files_per_class(files)

  classes = list(files_for_class.keys())[:num_classes]

  for cls in classes:
    new_files_for_class = files_for_class[cls]
    random.shuffle(new_files_for_class)
    files_for_class[cls] = new_files_for_class

  # Only use the number of classes you want in the dictionary
  files_for_class = {x: files_for_class[x] for x in list(files_for_class)[:num_classes]}

  dirs = {}
  for split_name, split_count in splits.items():
    print(split_name, ":")
    split_dir = download_dir / split_name
    split_files, files_for_class = split_class_lists(files_for_class, split_count)
    download_from_zip(zip_url, split_dir, split_files)
    dirs[split_name] = split_dir

  return dirs

def format_frames(frame, output_size):
  """
    Pad and resize an image from a video.

    Args:
      frame: Image that needs to resized and padded. 
      output_size: Pixel size of the output frame image.

    Return:
      Formatted frame with padding of specified output size.
  """
  frame = tf.image.convert_image_dtype(frame, tf.float32)
  frame = tf.image.resize_with_pad(frame, *output_size)
  return frame

def frames_from_video_file(video_path, n_frames, output_size = (224,224), frame_step = 15):
  """
    Creates frames from each video file present for each category.

    Args:
      video_path: File path to the video.
      n_frames: Number of frames to be created per video file.
      output_size: Pixel size of the output frame image.

    Return:
      An NumPy array of frames in the shape of (n_frames, height, width, channels).
  """
  # Read each video frame by frame
  result = []
  src = cv2.VideoCapture(str(video_path))  

  video_length = src.get(cv2.CAP_PROP_FRAME_COUNT)

  need_length = 1 + (n_frames - 1) * frame_step

  if need_length > video_length:
    start = 0
  else:
    max_start = video_length - need_length
    start = random.randint(0, max_start + 1)

  src.set(cv2.CAP_PROP_POS_FRAMES, start)
  # ret is a boolean indicating whether read was successful, frame is the image itself
  ret, frame = src.read()
  result.append(format_frames(frame, output_size))

  for _ in range(n_frames - 1):
    for _ in range(frame_step):
      ret, frame = src.read()
    if ret:
      frame = format_frames(frame, output_size)
      result.append(frame)
    else:
      result.append(np.zeros_like(result[0]))
  src.release()
  result = np.array(result)[..., [2, 1, 0]]

  return result

class FrameGenerator:
  def __init__(self, path, n_frames, training = False):
    """ Returns a set of frames with their associated label. 

      Args:
        path: Video file paths.
        n_frames: Number of frames. 
        training: Boolean to determine if training dataset is being created.
    """
    self.path = path
    self.n_frames = n_frames
    self.training = training
    self.class_names = sorted(set(p.name for p in self.path.iterdir() if p.is_dir()))
    self.class_ids_for_name = dict((name, idx) for idx, name in enumerate(self.class_names))

  def get_files_and_class_names(self):
    video_paths = list(self.path.glob('*/*.avi'))
    classes = [p.parent.name for p in video_paths] 
    return video_paths, classes

  def __call__(self):
    video_paths, classes = self.get_files_and_class_names()

    pairs = list(zip(video_paths, classes))

    if self.training:
      random.shuffle(pairs)

    for path, name in pairs:
      video_frames = frames_from_video_file(path, self.n_frames) 
      label = self.class_ids_for_name[name] # Encode labels
      yield video_frames, label

In [ ]:
URL = 'https://storage.googleapis.com/thumos14_files/UCF101_videos.zip'
download_dir = pathlib.Path('./UCF101_subset/')
subset_paths = download_ufc_101_subset(URL, 
                        num_classes = 10, 
                        splits = {"train": 30, "test": 20}, 
                        download_dir = download_dir)

Crie os datasets de treinamento e teste:

In [ ]:
batch_size = 8
num_frames = 8

output_signature = (tf.TensorSpec(shape = (None, None, None, 3), dtype = tf.float32),
                    tf.TensorSpec(shape = (), dtype = tf.int16))

train_ds = tf.data.Dataset.from_generator(FrameGenerator(subset_paths['train'], num_frames, training = True),
                                          output_signature = output_signature)
train_ds = train_ds.batch(batch_size)

test_ds = tf.data.Dataset.from_generator(FrameGenerator(subset_paths['test'], num_frames),
                                         output_signature = output_signature)
test_ds = test_ds.batch(batch_size)

Os rótulos gerados aqui representam a codificação das classes. Por exemplo, 'ApplyEyeMakeup' é mapeado para o inteiro. Confira os rótulos dos dados de treinamento para garantir que o dataset tenha sido misturado o suficiente. 

In [ ]:
for frames, labels in train_ds.take(10):
  print(labels)

Confira o formato dos dados.

In [ ]:
print(f"Shape: {frames.shape}")
print(f"Label: {labels.shape}")

## O que são MoViNets?

Conforme mencionado anteriormente, [MoViNets](https://arxiv.org/abs/2103.11511) são modelos para classificação de vídeos usados para inferência online ou de vídeos de streaming em diversas tarefas, como reconhecimento de ações. Você pode considerar o uso de MoViNets para classificar os dados dos seus vídeos a fim de fazer o reconhecimento de ações.

É eficiente e simples executar um classificador baseado em quadros 2D em vídeos inteiros ou quadro a quadro. Como o contexto temporal não é levado em conta, a exatidão é limitada, e as saídas geradas podem ser inconsistentes a cada quadro.

Uma CNN 3D simples usa contexto temporal bidirecional, o que pode aumentar a exatidão e a consistência temporal. Essas redes poderão exigir mais recursos e, como analisam o futuro, não podem ser usadas para fazer o streaming de dados.

![Standard convolution](https://www.tensorflow.org/images/tutorials/video/standard_convolution.png)

A arquitetura do MoViNet usa convoluções 3D que são "casuais" no eixo do tempo (como `layers.Conv1D` com `padding="causal"`). Isso traz algumas vantagens das duas estratégias, principalmente propiciar um streaming eficiente.

![Causal convolution](https://www.tensorflow.org/images/tutorials/video/causal_convolution.png)

A convolução casual garante que a saída no tempo *t* seja computada usando somente entradas até o tempo *t*. Para demonstrar como isso pode deixar o streaming mais eficiente, comece com um exemplo mais simples que você já deve conhecer: uma RNN, que passa o estado para frente do tempo:

![RNN model](https://www.tensorflow.org/images/tutorials/video/rnn_comparison.png)

In [ ]:
gru = layers.GRU(units=4, return_sequences=True, return_state=True)

inputs = tf.random.normal(shape=[1, 10, 8]) # (batch, sequence, channels)

result, state = gru(inputs) # Run it all at once

Ao definir o argumento `return_sequences=True` da RNN, você pede que ela retorne o estado no final da computação, o que permite pausar e depois continuar de onde parou, obtendo exatamente o mesmo resultado:

![States passing in RNNs](https://www.tensorflow.org/images/tutorials/video/rnn_state_passing.png)

In [ ]:
first_half, state = gru(inputs[:, :5, :])   # run the first half, and capture the state
second_half, _ = gru(inputs[:,5:, :], initial_state=state)  # Use the state to continue where you left off.

print(np.allclose(result[:, :5,:], first_half))
print(np.allclose(result[:, 5:,:], second_half))

As convoluções casuais podem ser usadas da mesma forma, se utilizadas com cuidado. Essa técnica foi usada no [Algoritmo de geração de Wavenet rápida](https://arxiv.org/abs/1611.09482) de Le Paine et al. No [artigo do MoViNet](https://arxiv.org/abs/2103.11511), o `state` (estado) é chamado de "Buffer de stream".

![States passed in causal convolution](https://www.tensorflow.org/images/tutorials/video/causal_conv_states.png)

Ao passar esse pequeno estado para frente, é possível evitar o recálculo de todo o campo receptivo exibido acima. 

## Baixe um modelo MoViNet pré-treinado

Nesta seção, você vai:

1. Criar um modelo MoViNet usando o código aberto fornecido em [`official/projects/movinet`](https://github.com/tensorflow/models/tree/master/official/projects/movinet) dos modelos do TensorFlow.
2. Carregar os pesos pré-treinados.
3. Congelar a base convolucional ou todas as outras camadas, exceto o cabeçalho final do classificador, para acelerar o ajuste fino.

Para criar o modelo, você pode começar com a configuração `a0`, pois é a mais rápida para o treinamento em comparação com outros modelos. Confira os [modelos MoViNet disponíveis no TensorFlow Model Garden](https://github.com/tensorflow/models/blob/master/official/projects/movinet/configs/movinet.py) para descobrir o que funcionará para seu caso de uso.

In [ ]:
model_id = 'a0'
resolution = 224

tf.keras.backend.clear_session()

backbone = movinet.Movinet(model_id=model_id)
backbone.trainable = False

# Set num_classes=600 to load the pre-trained weights from the original model
model = movinet_model.MovinetClassifier(backbone=backbone, num_classes=600)
model.build([None, None, None, None, 3])

# Load pre-trained weights
!wget https://storage.googleapis.com/tf_model_garden/vision/movinet/movinet_a0_base.tar.gz -O movinet_a0_base.tar.gz -q
!tar -xvf movinet_a0_base.tar.gz

checkpoint_dir = f'movinet_{model_id}_base'
checkpoint_path = tf.train.latest_checkpoint(checkpoint_dir)
checkpoint = tf.train.Checkpoint(model=model)
status = checkpoint.restore(checkpoint_path)
status.assert_existing_objects_matched()

Para criar um classificador, crie uma função que receba o backbone e o número de classes em um dataset. A função `build_classifier` fará isso. Neste caso, o novo classificador receberá `num_classes` saídas (10 classes para este subconjunto do UCF101).

In [ ]:
def build_classifier(batch_size, num_frames, resolution, backbone, num_classes):
  """Builds a classifier on top of a backbone model."""
  model = movinet_model.MovinetClassifier(
      backbone=backbone,
      num_classes=num_classes)
  model.build([batch_size, num_frames, resolution, resolution, 3])

  return model

In [ ]:
model = build_classifier(batch_size, num_frames, resolution, backbone, 10)

Neste tutorial, escolha o otimizador `tf.keras.optimizers.Adam` e a função de perda `tf.keras.losses.SparseCategoricalCrossentropy`. Use o argumento metrics para ver a exatidão do desempenho do modelo a cada passo.

In [ ]:
num_epochs = 2

loss_obj = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001)

model.compile(loss=loss_obj, optimizer=optimizer, metrics=['accuracy'])

Treine o modelo. Após duas épocas, observe uma perda baixa com exatidão alta para o dataset de treinamento e de teste. 

In [ ]:
results = model.fit(train_ds,
                    validation_data=test_ds,
                    epochs=num_epochs,
                    validation_freq=1,
                    verbose=1)

## Avalie o modelo

O modelo atingiu uma exatidão alta para o dataset de treinamento. Em seguida, use `Model.evaluate` do Keras para avaliar a exatidão para o conjunto de teste.

In [ ]:
model.evaluate(test_ds, return_dict=True)

Para avaliar o desempenho do modelo com mais detalhes, use uma [matriz de confusão](https://www.tensorflow.org/api_docs/python/tf/math/confusion_matrix), que permite avaliar o desempenho do modelo de classificação além de apenas a exatidão. Para criar a matriz de confusão para este problema de classificação multiclasse, obtenha os valores reais do conjunto de teste e os valores previstos.

In [ ]:
def get_actual_predicted_labels(dataset):
  """
    Create a list of actual ground truth values and the predictions from the model.

    Args:
      dataset: An iterable data structure, such as a TensorFlow Dataset, with features and labels.

    Return:
      Ground truth and predicted values for a particular dataset.
  """
  actual = [labels for _, labels in dataset.unbatch()]
  predicted = model.predict(dataset)

  actual = tf.stack(actual, axis=0)
  predicted = tf.concat(predicted, axis=0)
  predicted = tf.argmax(predicted, axis=1)

  return actual, predicted

In [ ]:
def plot_confusion_matrix(actual, predicted, labels, ds_type):
  cm = tf.math.confusion_matrix(actual, predicted)
  ax = sns.heatmap(cm, annot=True, fmt='g')
  sns.set(rc={'figure.figsize':(12, 12)})
  sns.set(font_scale=1.4)
  ax.set_title('Confusion matrix of action recognition for ' + ds_type)
  ax.set_xlabel('Predicted Action')
  ax.set_ylabel('Actual Action')
  plt.xticks(rotation=90)
  plt.yticks(rotation=0)
  ax.xaxis.set_ticklabels(labels)
  ax.yaxis.set_ticklabels(labels)

In [ ]:
fg = FrameGenerator(subset_paths['train'], num_frames, training = True)
label_names = list(fg.class_ids_for_name.keys())

In [ ]:
actual, predicted = get_actual_predicted_labels(test_ds)
plot_confusion_matrix(actual, predicted, label_names, 'test')

## Próximos passos

Agora que você já conhece um pouco o modelo MoViNet e como usar diversas APIs do TensorFlow (para aprendizado por transferência, por exemplo), tente usar o código neste tutorial com seu próprio dataset. Os dados não precisam estar limitados a vídeos. Dados volumétricos, como tomografias, também podem ser usados com CNNs 3D. Os datasets NUSDAT e IMH mencionados em [Redes Neurais Convolucionais 3D baseadas em tomografias do cérebro para classificação de esquizofrenia e controles](https://arxiv.org/pdf/2003.08818.pdf) podem ser duas fontes de dados de tomografia.

Especificamente, você pode usar a classe `FrameGenerator` utilizada neste tutorial e os outros dados de vídeos e tutoriais de classificação para ajudar a carregar os dados em seus modelos.

Para saber mais sobre como trabalhar com dados de vídeo no TensorFlow, confira os tutoriais a seguir:

- [Carregue dados de vídeo](https://www.tensorflow.org/tutorials/load_data/video)
- [Crie um modelo CNN 3D para a classificação de vídeos](https://www.tensorflow.org/tutorials/video/video_classification)
- [MoViNet para o reconhecimento de ações de streaming](https://www.tensorflow.org/hub/tutorials/movinet)